# H(curl) and H(div) function spaces (Unit 2.3)

Scalar and vectorial elements in NGSolve:

Standard continuous $H^1$ elements:
<img src='resources/nodalelement.png' width='100px' />

Nedelec's tangentially-continuous $H(curl)$-conforming edge elements:
<img src='resources/edgeelement.png'  width='100px' />

Raviart-Thomas normally-continuous $H(div)$-conforming face elements:
<img src='resources/faceelement.png'  width='100px' />

Discontinuous $L_2$ elements:
<img src='resources/l2element.png'  width='100px' />



These vector-valued spaces allow to represent physical quantities which are either normally or tangentially continuous.

The finite element spaces are related by the de Rham complex:

$$\begin{array}[ccccccc]
\ H^1 & \underset{\rightarrow}{\text{grad}} & H(curl) & \underset{\rightarrow}{\text{curl}} & H(div) & \underset{\rightarrow}{\text{div}} & L_2\\
\bigcup & \ & \bigcup & \ & \bigcup & \ & \bigcup \\ 
W_h & \underset{\rightarrow}{\text{grad}} & V_h & \underset{\rightarrow}{\text{curl}} & Q_h & \underset{\rightarrow}{\text{div}} & S_h
\end{array}$$
NGSolve supports these elements of arbitrary order, on all common element shapes (trigs, quads, tets, prisms, pyramids, hexes). Elements may be curved.

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square
from netgen.csg import unit_cube
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))


Generate a higher order $H^1$ space. We first explore its different types of basis functions.

In [2]:
order=3
fes = H1(mesh, order=order)
gfu = GridFunction(fes)
Draw(gfu)

The first #vertices basis functions are hat-functions. By setting the solution vector to a unit-vector, we may look at the individual basis functions:

In [3]:
SetVisualization(min=0, max=1)
gfu.vec[:] = 0
# vertex nr:
gfu.vec[18] = 1
Redraw()


The next are edge-bubbles, where we have (order−1) basis functions per edge. A NodeId object refers to a particular vertex, edge, face or cell node in the mesh. We can ask for the degrees of freedom on a node:

In [4]:
# basis functions on edge nr:
edge_dofs = fes.GetDofNrs(NodeId(EDGE,10))
print("edge_dofs =", edge_dofs)
SetVisualization(min=-0.05, max=0.05)
gfu.vec[:] = 0
gfu.vec[edge_dofs[1]] = 1
Redraw()


edge_dofs = (44, 45)


Finally, we have $(p−1)(p−2)/2$
inner basis functions on every triangle:

In [5]:
trig_dofs = fes.GetDofNrs(NodeId(FACE,0))
print("trig_dofs = ", trig_dofs)
SetVisualization(min=0, max=0.03)
gfu.vec[:] = 0
gfu.vec[trig_dofs[0]] = 1
Redraw()


trig_dofs =  (130,)


The FESpace also maintains information about local dofs, interface dofs and wire-basket dofs for the BDDC preconditioner:

In [6]:
for i in range(fes.ndof):
    print (i,":", fes.CouplingType(i))

0 : COUPLING_TYPE.WIREBASKET_DOF
1 : COUPLING_TYPE.WIREBASKET_DOF
2 : COUPLING_TYPE.WIREBASKET_DOF
3 : COUPLING_TYPE.WIREBASKET_DOF
4 : COUPLING_TYPE.WIREBASKET_DOF
5 : COUPLING_TYPE.WIREBASKET_DOF
6 : COUPLING_TYPE.WIREBASKET_DOF
7 : COUPLING_TYPE.WIREBASKET_DOF
8 : COUPLING_TYPE.WIREBASKET_DOF
9 : COUPLING_TYPE.WIREBASKET_DOF
10 : COUPLING_TYPE.WIREBASKET_DOF
11 : COUPLING_TYPE.WIREBASKET_DOF
12 : COUPLING_TYPE.WIREBASKET_DOF
13 : COUPLING_TYPE.WIREBASKET_DOF
14 : COUPLING_TYPE.WIREBASKET_DOF
15 : COUPLING_TYPE.WIREBASKET_DOF
16 : COUPLING_TYPE.WIREBASKET_DOF
17 : COUPLING_TYPE.WIREBASKET_DOF
18 : COUPLING_TYPE.WIREBASKET_DOF
19 : COUPLING_TYPE.WIREBASKET_DOF
20 : COUPLING_TYPE.WIREBASKET_DOF
21 : COUPLING_TYPE.WIREBASKET_DOF
22 : COUPLING_TYPE.WIREBASKET_DOF
23 : COUPLING_TYPE.WIREBASKET_DOF
24 : COUPLING_TYPE.INTERFACE_DOF
25 : COUPLING_TYPE.INTERFACE_DOF
26 : COUPLING_TYPE.INTERFACE_DOF
27 : COUPLING_TYPE.INTERFACE_DOF
28 : COUPLING_TYPE.INTERFACE_DOF
29 : COUPLING_TYPE.INTERFACE_

In NGSolve we use hierarchical high order finite element basis functions with node-wise exact sequences. The lowest order space $W_{l.o}

is the edge-element space:
$$W_{hp} = W_{p=1} + \sum_E W_E + \sum_F W_F + \sum_C W_C \subset H^1$$
$$V_{hp} = W_{l.o} + \sum_E V_E + \sum_F V_F + \sum_C V_C \subset H(curl)$$

where the edge, face and cell blocks are compatible in the sense that
$$\nabla W_E = V_E, \nabla W_F \subset V_F, \nabla W_C \subset V_C$$

We obtain this by using gradients of $H^1$ basis functions as $H(curl)$ basis functions, and some more (see thesis Sabine Zaglmayr):
$$V_E = \text{span}\lbrace \nabla \phi_{E,i}^{H^1}\rbrace $$

$$V_F = \text{span}\lbrace \nabla \phi_{F,i}^{H^1} \cup  \tilde{\phi}_{F,i}^{H(curl)}\rbrace $$

$$V_C = \text{span}\lbrace \nabla \phi_{C,i}^{H^1} \cup  \tilde{\phi}_{C,i}^{H(curl)}\rbrace $$



In [8]:
fes = HCurl(mesh, order=2)
uc = GridFunction(fes, name="uc")
Draw (uc)
Draw (curl(uc), mesh, "curl")


In [9]:
edge_dofs = fes.GetDofNrs(NodeId(EDGE,10))
print ("edgedofs: ", edge_dofs)
uc.vec[:] = 0
uc.vec[edge_dofs[1]] = 1
Redraw()


edgedofs:  (10, 73, 74)


In [ ]:
look at them by activating Draw Surface Vectors.

In [10]:
face_dofs = fes.GetDofNrs(NodeId(FACE,10))
print ("facedofs: ", face_dofs)
uc.vec[:] = 0
uc.vec[face_dofs[1]] = 1
Redraw()

facedofs:  (234, 235, 236)
